In [ ]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.functional as f 
from torch.autograd import Variable
torch.backends.cudnn.deterministic = True
import pandas as pd
import numpy as np
import torch.utils.data as data_utils
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split
from sklearn import metrics
import random 
from matplotlib import pyplot as plt
from sklearn import decomposition
from sklearn import manifold
from scipy import stats
from itertools import cycle
import sys
import utilities as ut
from Network import Net
import Network as nn
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True


results = []

n = 5000#train_dataset.shape[0] 
epsilon = 0.1

hidden_size = 500
hidden_size2 = 500
num_classes = 2
num_epochs = 200
batch_size = 128
learning_rate = 0.001
learning_rate2 = 0.001
regularization = False
add_DR_based_data = True

samples = 10000

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True



#for epsilon in [0.1, 0.05, 0.025, 0.15]:
for batch_size in [256]:
    for hidden_size in [100]:
        for aux_loss_activated in [True]:
            for EPS1 in [0.1]:
                for n in [100000]:
                    for opt in [1]:
                        for t in range(1):
                            train_dataset, test_dataset = ut.load_files(dataset=1)
                            input_size = train_dataset.shape[1]-1
                            

                            if n < 50000:
                                train_dataset = ut.down_sampling(train_dataset)
                                train_dataset = train_dataset.sample(n)
                                print(train_dataset)
                            else: 
                                trainig_dataset_a = train_dataset[train_dataset.label=='ClassA']
                                print('shape: ', trainig_dataset_a.shape[0])
                                n2 = n - trainig_dataset_a.shape[0]
                                print('clase no RR Lrae', n2)
                                trainig_dataset_b = train_dataset[~(train_dataset.label=='ClassA')].sample(n2)
                                train_dataset = pd.concat([trainig_dataset_a, trainig_dataset_b])
                            
                            train_dataset, test_dataset = ut.delete_outliers(train_dataset, test_dataset)

                            train_dataset = ut.sort_columns(train_dataset)
                            test_dataset = ut.sort_columns(test_dataset)

                            test_dataset_pred = test_dataset.copy()
                            train_dataset_pred = train_dataset.copy()

                            try:
                                data_prior = ut.generate_samples_2D(samples, train_dataset)

                                train_dataset, val_dataset = train_test_split(train_dataset, test_size=0.2, random_state=42)

                                train_dataset_prior, val_dataset_prior = train_test_split(data_prior, test_size=0.2, random_state=42)
                                print(train_dataset_prior.columns)
                                _, _, train_target_prior, train_loader_prior = ut.get_tensors(train_dataset_prior, batch_size)
                                _, _, val_target_prior, val_loader_prior     = ut.get_tensors(val_dataset_prior, batch_size)
                                _, _, train_target, train_loader             = ut.get_tensors(train_dataset, batch_size)
                                _, _, train_target_pred, train_loader_pred   = ut.get_tensors(train_dataset_pred, batch_size)
                                _, _, val_target, val_loader                 = ut.get_tensors(val_dataset_prior, batch_size)
                                _, _, test_target, test_loader               = ut.get_tensors(test_dataset, batch_size)
                                _, _, test_target_pred, test_loader_pred     = ut.get_tensors(test_dataset_pred, batch_size)

                                net = Net(input_size, hidden_size, hidden_size, num_classes)
                                net.cuda()

                                hist_val, hist_train = nn.train(net, train_loader, train_loader_prior, val_loader, test_loader,
                                EPS1, learning_rate, input_size, aux_loss_activated=aux_loss_activated)

                                acc_train = nn.get_results(net, train_loader, input_size)
                                acc_test =nn.get_results(net, test_loader, input_size)
                                results.append([acc_train, acc_test, epsilon, batch_size, hidden_size, aux_loss_activated, EPS1, n, opt])
                                pd.DataFrame(results, columns=['acc_train', 'acc_test', 'epsilon', 'batch_size', 'hidden_size',
                                    'aux_loss_activated', 'EPS1', 'n', 'opt']).to_csv('test_2d.csv')
                            except: 
                                print(str(epsilon)+"-"+str(batch_size)+"-"+str(hidden_size)+"-"+str(aux_loss_activated)+"-"+str(EPS1))

/home/franciscoperez/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.8' currently installed).
  warnings.warn(msg, UserWarning)
In /home/franciscoperez/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/franciscoperez/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/franciscoperez/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/franciscoperez/anacon

Freq1_harmonics_rel_phase_0
Freq2_harmonics_rel_phase_0
Freq3_harmonics_rel_phase_0
shape:  28847
clase no RR Lrae 71153
(78021, 61)
(28625, 60)
(28625, 61)
Index(['PeriodLS', 'Amplitude', 'AndersonDarling', 'Autocor_length',
       'Beyond1Std', 'CAR_mean', 'CAR_sigma', 'CAR_tau', 'Con', 'Eta_e',
       'FluxPercentileRatioMid20', 'FluxPercentileRatioMid35',
       'FluxPercentileRatioMid50', 'FluxPercentileRatioMid65',
       'FluxPercentileRatioMid80', 'Freq1_harmonics_amplitude_0',
       'Freq1_harmonics_amplitude_1', 'Freq1_harmonics_amplitude_2',
       'Freq1_harmonics_amplitude_3', 'Freq1_harmonics_rel_phase_1',
       'Freq1_harmonics_rel_phase_2', 'Freq1_harmonics_rel_phase_3',
       'Freq2_harmonics_amplitude_0', 'Freq2_harmonics_amplitude_1',
       'Freq2_harmonics_amplitude_2', 'Freq2_harmonics_amplitude_3',
       'Freq2_harmonics_rel_phase_1', 'Freq2_harmonics_rel_phase_2',
       'Freq2_harmonics_rel_phase_3', 'Freq3_harmonics_amplitude_0',
       'Freq3_harmonics_am

/home/franciscoperez/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


shape tensor:  torch.Size([78021, 60])
____get_tensor_function____
shape tensor:  torch.Size([4000, 60])
____get_tensor_function____
shape tensor:  torch.Size([22840, 60])
____get_tensor_function____
shape tensor:  torch.Size([22840, 60])
Epochs:  10000
Epoch:  0
training: epoch:  1  loss:  537.3232112526894 -- aux loss:  921.436275578104
validating: epoch:  1  loss:  0.009292953996919096
Epoch:  1
training: epoch:  2  loss:  671.9476127624512 -- aux loss:  1.3583868369460106
validating: epoch:  2  loss:  1.9798427842943056e-05
Epoch:  2
training: epoch:  3  loss:  554.8535816669464 -- aux loss:  0.0017952769994735718
validating: epoch:  3  loss:  0.0
Epoch:  3
training: epoch:  4  loss:  481.8148113489151 -- aux loss:  1.821666955947876e-06
validating: epoch:  4  loss:  0.0
Epoch:  4
training: epoch:  5  loss:  403.1152000427246 -- aux loss:  1.9120052456855774e-06
validating: epoch:  5  loss:  0.0
Epoch:  5
training: epoch:  6  loss:  332.0632518529892 -- aux loss:  1.600012183189392

training: epoch:  48  loss:  23.38239026069641 -- aux loss:  6.612390279769897e-08
validating: epoch:  48  loss:  0.0
Epoch:  48
training: epoch:  49  loss:  23.00497880578041 -- aux loss:  6.984919309616089e-08
validating: epoch:  49  loss:  0.0
Epoch:  49
training: epoch:  50  loss:  22.670318361371756 -- aux loss:  7.916241884231567e-08
validating: epoch:  50  loss:  0.0
The current loss: 0.0
the_last_loss: 0.0
trigger times: 0
Accuracy of the network on test objects: 97 %
97
Accuracy of the network on test objects: 100 %
100
Accuracy of the network on test objects: 70 %
70
sum mask2 - L1:  tensor(1314, device='cuda:0')
sum mask2 - L2:  tensor(0, device='cuda:0')
sum mask2 - L3:  tensor(0, device='cuda:0')
sum mask1 - L1:  tensor(4686, device='cuda:0')
sum mask1 - L2:  tensor(10000, device='cuda:0')
sum mask1 - L3:  tensor(200, device='cuda:0')
Epoch:  50
training: epoch:  51  loss:  22.37931204214692 -- aux loss:  8.381903171539307e-08
validating: epoch:  51  loss:  0.0
Epoch:  51


...

training: epoch:  4060  loss:  3.2333646044135094 -- aux loss:  0.0
validating: epoch:  4060  loss:  0.0
The current loss: 0.0
the_last_loss: 0.0
trigger times: 0
Accuracy of the network on test objects: 99 %
99
Accuracy of the network on test objects: 100 %
100
Accuracy of the network on test objects: 85 %
85
sum mask2 - L1:  tensor(1314, device='cuda:0')
sum mask2 - L2:  tensor(0, device='cuda:0')
sum mask2 - L3:  tensor(0, device='cuda:0')
sum mask1 - L1:  tensor(4686, device='cuda:0')
sum mask1 - L2:  tensor(10000, device='cuda:0')
sum mask1 - L3:  tensor(200, device='cuda:0')
Epoch:  4060
training: epoch:  4061  loss:  3.2323848605155945 -- aux loss:  0.0
validating: epoch:  4061  loss:  0.0
Epoch:  4061
training: epoch:  4062  loss:  3.232717990875244 -- aux loss:  0.0
validating: epoch:  4062  loss:  0.0
Epoch:  4062
training: epoch:  4063  loss:  3.232913978397846 -- aux loss:  0.0
validating: epoch:  4063  loss:  0.0
Epoch:  4063
training: epoch:  4064  loss:  3.23189836740493

training: epoch:  4110  loss:  3.2178736701607704 -- aux loss:  0.0
validating: epoch:  4110  loss:  0.0
The current loss: 0.0
the_last_loss: 0.0
trigger times: 0
Accuracy of the network on test objects: 99 %
99
Accuracy of the network on test objects: 100 %
100
Accuracy of the network on test objects: 85 %
85
sum mask2 - L1:  tensor(1314, device='cuda:0')
sum mask2 - L2:  tensor(0, device='cuda:0')
sum mask2 - L3:  tensor(0, device='cuda:0')
sum mask1 - L1:  tensor(4686, device='cuda:0')
sum mask1 - L2:  tensor(10000, device='cuda:0')
sum mask1 - L3:  tensor(200, device='cuda:0')
Epoch:  4110
training: epoch:  4111  loss:  3.219801813364029 -- aux loss:  0.0
validating: epoch:  4111  loss:  0.0
Epoch:  4111
training: epoch:  4112  loss:  3.2185959070920944 -- aux loss:  0.0
validating: epoch:  4112  loss:  0.0
Epoch:  4112
training: epoch:  4113  loss:  3.2182583659887314 -- aux loss:  0.0
validating: epoch:  4113  loss:  0.0
Epoch:  4113
training: epoch:  4114  loss:  3.2172091454267

training: epoch:  4160  loss:  3.2047734409570694 -- aux loss:  0.0
validating: epoch:  4160  loss:  0.0
The current loss: 0.0
the_last_loss: 0.0
trigger times: 0
Accuracy of the network on test objects: 99 %
99
Accuracy of the network on test objects: 100 %
100
Accuracy of the network on test objects: 85 %
85
sum mask2 - L1:  tensor(1314, device='cuda:0')
sum mask2 - L2:  tensor(0, device='cuda:0')
sum mask2 - L3:  tensor(0, device='cuda:0')
sum mask1 - L1:  tensor(4686, device='cuda:0')
sum mask1 - L2:  tensor(10000, device='cuda:0')
sum mask1 - L3:  tensor(200, device='cuda:0')
Epoch:  4160
training: epoch:  4161  loss:  3.204831227660179 -- aux loss:  0.0
validating: epoch:  4161  loss:  0.0
Epoch:  4161
training: epoch:  4162  loss:  3.2038736268877983 -- aux loss:  0.0
validating: epoch:  4162  loss:  0.0
Epoch:  4162
training: epoch:  4163  loss:  3.20394803583622 -- aux loss:  0.0
validating: epoch:  4163  loss:  0.0
Epoch:  4163
training: epoch:  4164  loss:  3.202379629015922

training: epoch:  4210  loss:  3.192903868854046 -- aux loss:  0.0
validating: epoch:  4210  loss:  0.0
The current loss: 0.0
the_last_loss: 0.0
trigger times: 0
Accuracy of the network on test objects: 99 %
99
Accuracy of the network on test objects: 100 %
100
Accuracy of the network on test objects: 85 %
85
sum mask2 - L1:  tensor(1314, device='cuda:0')
sum mask2 - L2:  tensor(0, device='cuda:0')
sum mask2 - L3:  tensor(0, device='cuda:0')
sum mask1 - L1:  tensor(4686, device='cuda:0')
sum mask1 - L2:  tensor(10000, device='cuda:0')
sum mask1 - L3:  tensor(200, device='cuda:0')
Epoch:  4210
training: epoch:  4211  loss:  3.191598169505596 -- aux loss:  0.0
validating: epoch:  4211  loss:  0.0
Epoch:  4211
training: epoch:  4212  loss:  3.1912836357951164 -- aux loss:  0.0
validating: epoch:  4212  loss:  0.0
Epoch:  4212
training: epoch:  4213  loss:  3.1908593252301216 -- aux loss:  0.0
validating: epoch:  4213  loss:  0.0
Epoch:  4213
training: epoch:  4214  loss:  3.19163981825113

training: epoch:  4260  loss:  3.1799939051270485 -- aux loss:  0.0
validating: epoch:  4260  loss:  0.0
The current loss: 0.0
the_last_loss: 0.0
trigger times: 0
Accuracy of the network on test objects: 99 %
99
Accuracy of the network on test objects: 100 %
100
Accuracy of the network on test objects: 85 %
85
sum mask2 - L1:  tensor(1314, device='cuda:0')
sum mask2 - L2:  tensor(0, device='cuda:0')
sum mask2 - L3:  tensor(0, device='cuda:0')
sum mask1 - L1:  tensor(4686, device='cuda:0')
sum mask1 - L2:  tensor(10000, device='cuda:0')
sum mask1 - L3:  tensor(200, device='cuda:0')
Epoch:  4260
training: epoch:  4261  loss:  3.1787713766098022 -- aux loss:  0.0
validating: epoch:  4261  loss:  0.0
Epoch:  4261
training: epoch:  4262  loss:  3.1942741200327873 -- aux loss:  0.0
validating: epoch:  4262  loss:  0.0
Epoch:  4262
training: epoch:  4263  loss:  3.180070646107197 -- aux loss:  0.0
validating: epoch:  4263  loss:  0.0
Epoch:  4263
training: epoch:  4264  loss:  3.1782416105270

In [ ]:
#plt.plot(aux_loss_behaviour)
#plt.show()

In [ ]:
nn.get_results(net, train_loader, input_size)
nn.get_results(net, test_loader, input_size)

In [ ]:
torch.save(net.state_dict(), 'model.pkl')

In [ ]:
plt.plot(hist_val, label ='validation')
plt.plot(hist_train, label ='train')
plt.legend()
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.savefig('images/'+str(samples)+'_'+str(epsilon)+'_'+str(n)+"_"+str(hidden_size)+"_Loss_Training.png")
plt.show()

In [ ]:
#csv_file = open("size_MLP_noise.csv", "a")
#csv_file.write(str(np.asarray(acc_testing))+","+str(np.asarray(acc_training))+","+str(samples)+","+str(epsilon)+","+str(n)+","+str(hidden_size)+"\n")
#csv_file.close()

In [ ]:
outputs, intermediates, intermediates2, labels = ut.get_representations(net, train_loader, device)
outputs_test, intermediates_test, intermediates2_test, labels_test = ut.get_representations(net, test_loader, device)

In [ ]:
intermediate_pca_data, intermediate_pca_data_test = ut.get_pca(intermediates, data_test=intermediates_test)
fig, axs = plt.subplots(2, 3, figsize=(20,15))
ut.plot_representations(intermediate_pca_data, labels, axs[0, 0])
ut.plot_representations(intermediate_pca_data_test, labels_test, axs[1, 0])
intermediate2_pca_data, intermediate2_pca_data_test = ut.get_pca(intermediates2, data_test=intermediates2_test)
ut.plot_representations(intermediate2_pca_data, labels, axs[0, 1])
ut.plot_representations(intermediate2_pca_data_test, labels_test, axs[1, 1])
output_pca_data, output_pca_data_test = ut.get_pca(outputs, data_test=outputs_test)
ut.plot_representations(output_pca_data, labels, axs[0, 2])
ut.plot_representations(output_pca_data_test, labels_test, axs[1, 2])

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(20,15))
print('first')
reducer = umap.UMAP()
print('first ..')
mapper_train = reducer.fit_transform(intermediates.numpy())
print('first ...')
reducer = umap.UMAP()
mapper_test = reducer.fit_transform(intermediates_test.numpy())
ut.plot_representations(mapper_train, labels, axs[0, 0])
ut.plot_representations(mapper_test, labels_test, axs[1, 0])

reducer = umap.UMAP()
mapper_train = reducer.fit_transform(intermediates2.numpy())
reducer = umap.UMAP()
mapper_test = reducer.fit_transform(intermediates2_test.numpy())
ut.plot_representations(mapper_train, labels, axs[0, 1])
ut.plot_representations(mapper_test, labels_test, axs[1, 1])

reducer = umap.UMAP()
mapper_train = reducer.fit_transform(outputs.numpy())
reducer = umap.UMAP()
mapper_test = reducer.fit_transform(outputs_test.numpy())

ut.plot_representations(mapper_train, labels, axs[0, 2])
ut.plot_representations(mapper_test, labels_test, axs[1, 2])

In [ ]:
N_CURVES = 25000

fig, axs = plt.subplots(2, 3, figsize=(20,15))
intermediate_tsne_data, intermediate_tsne_data_test = ut.get_tsne(intermediates, data_test= intermediates_test, n_curves = N_CURVES)
ut.plot_representations(intermediate_tsne_data, labels, axs[0, 0],  n_curves = N_CURVES)
ut.plot_representations(intermediate_tsne_data_test, labels_test, axs[1, 0], n_curves = N_CURVES)

intermediate2_tsne_data, intermediate2_tsne_data_test = ut.get_tsne(intermediates2, data_test=intermediates2_test, n_curves = N_CURVES)
ut.plot_representations(intermediate2_tsne_data, labels, axs[0, 1], n_curves = N_CURVES)
ut.plot_representations(intermediate2_tsne_data_test, labels_test, axs[1, 1], n_curves = N_CURVES)

output_tsne_data, output2_tsne_data_test = ut.get_tsne(outputs, data_test=outputs_test, n_curves = N_CURVES)
ut.plot_representations(output_tsne_data, labels, axs[0, 2], n_curves = N_CURVES)
ut.plot_representations(output2_tsne_data_test, labels_test, axs[1, 2], n_curves = N_CURVES)

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15,15))
curves, labels, probs_train = ut.get_predictions(net, train_loader_pred, device)
pred_labels = probs_train.argmax(1, keepdim = True)
ut.plot_confusion_matrix(np.round(labels), pred_labels, ax1)
curves, labels, probs_test = ut.get_predictions(net, test_loader_pred, device)
pred_labels = probs_test.argmax(1, keepdim = True)
ut.plot_confusion_matrix(np.round(labels), pred_labels, ax2)

In [ ]:
curves, labels, probs_train_sample = ut.get_predictions(net, train_loader, device)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20,15))
ax1.hist(probs_train[:,0], color='black')
ax1.set_xlabel('training set')
ax2.hist(probs_train_sample[:,0], color='black')
ax2.set_xlabel('training set + samples')
ax3.hist(probs_test[:,0], color='black')
ax3.set_xlabel('testing set')
plt.show()

In [ ]:
#N_WEIGHTS = 25
#weights = net.fc2.weight.data
#plot_weights(weights, N_WEIGHTS)

In [ ]:
weights1 = net.fc1.weight.data
w1 = weights1.cpu().numpy().reshape(-1,1)
weights2 = net.fc2.weight.data
w2 = weights2.cpu().numpy().reshape(-1,1)
weights3 = net.fc3.weight.data
w3 = weights3.cpu().numpy().reshape(-1,1)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20,15))
ax1.hist(w1, color='black')
ax1.set_xlabel('Layer 1')
ax2.hist(w2, color='black')
ax2.set_xlabel('Layer 2')
ax3.hist(w3, color='black')
ax3.set_xlabel('Layer 3')
plt.show()